In [1]:
import sys
sys.path.append("..")
#from motion.longitudinal_controller import LongitudinalController
from carlasim.carla_client import CarlaClient
from carlasim.ego_car import EgoCar

import gi
gi.require_version('Gst', '1.0')
gi.require_version('GstRtspServer', '1.0')
from gi.repository import Gst, GLib
Gst.init(None)

/workspaces/crawler-poc/notebooks/../carlasim/video_streamer.py:1: PyGIWarning: Gst was imported without specifying a version first. Use gi.require_version('Gst', '1.0') before import to ensure that the right version gets loaded.
  from gi.repository import Gst


[]

Setup the simulation environment

In [2]:
StartPosition = ['Town07', 0, 0, 0, 0.0]
DesiredStableSpeed = 20

Carla simulation run

In [3]:
client = CarlaClient(town=StartPosition[0])
ego = EgoCar(client)
ego.set_pose(StartPosition[1], StartPosition[2], StartPosition[3], StartPosition[4])
ego.set_power(0)
ego.set_steering(0)

new power: 0


In [6]:
import time
import threading
import numpy as np


class DiscreteComponent:
    thr: threading.Thread
    _period_ms: int
    _run: bool
    _last_timestamp: float


    def __init__(self, period_ms: int) -> None:
        self._period_ms = period_ms / 1000
        self.thr = None
        self._run = False
        
    def start (self) -> None:
        if self._run:
            self.destroy()

        self._run = True
        self.thr = threading.Thread(target=self.__discrete_loop)
        self.thr.start()

    def destroy(self) -> None:
        self._run = False

        if self.thr is None:
            return
        
        self.thr = None

    def __discrete_loop(self):
        self._last_timestamp = time.time()
        
        while (self._run):
            t = time.time()
            dt = t - self._last_timestamp
            self._loop(dt)
            self._last_timestamp = t
            time.sleep(self._period_ms)


    def _loop(self, dt: float) -> None:
        pass


class LongitudinalController(DiscreteComponent):
    __KP = 1.0
    __KI = 0.2
    __KD = 0.01

    _odometer: callable
    _power_actuator : callable
    _brake_actuator : callable
    _error_I: float
    _error_D: float
    _desired_speed: float
    _prev_throttle: float
   
    def __init__(self, sampling_period_ms: int, odometer: callable, power_actuator: callable, brake_actuator: callable) -> None:
        super().__init__(sampling_period_ms)
        self._desired_speed = 0.0
        self._prev_throttle = 0.0
        self._odometer = odometer
        self._power_actuator = power_actuator
        self._brake_actuator = brake_actuator
        self._error_I = 0.0
        self._error_D = 0.0

    def brake(self, brake_strenght: float) -> None:
        self._power_actuator(0)
        self._brake_actuator(brake_strenght)
    
    def _loop(self, dt: float) -> None:
        current_speed = self._odometer()
        error = self._desired_speed - current_speed

        # Autobreak
        if current_speed == 0 and self._desired_speed == 0:
            self.brake(1.0)
            return

        self._error_I += error * dt
        self._error_D = (error - self._error_D) / dt
        acc = LongitudinalController.__KP * error \
                    + LongitudinalController.__KI * self._error_I\
                    + LongitudinalController.__KD * self._error_D
        
        if acc < 0:
            self.brake(0.5)
            return

        if acc == 0:
            self.brake(0.0)
            return
        
        throttle = 240 * (np.tanh(acc) + 1)/2
        if throttle - self._prev_throttle > 24:
            throttle = self._prev_throttle + 24
        
        self._prev_throttle = throttle
        self._brake_actuator(0)         
        self._power_actuator(throttle)
      

    def set_speed(self, desired_speed: float) -> None:
        self._desired_speed = desired_speed

    def destroy(self) -> None:
        if self._desired_speed != 0:
            self._desired_speed = 0.0

        self.brake(1.0)
        super().destroy()

new power: 0


Longitudinal controller testing

In [5]:
controller = LongitudinalController(250, 
                                    odometer=lambda : ego.odometer.read(), 
                                    power_actuator=lambda power: ego.set_power(power),
                                    brake_actuator=lambda brake: ego.set_brake(brake))

controller.set_speed(20.0)
controller.start()

In [8]:
controller.set_speed(20)

new power: 0
new power: 240.0
new power: 240.0


new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 0
new power: 240

In [9]:
controller.destroy()

new power: 240.0
new power: 0
new power: 0


new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: 240.0
new power: 0
new power: